In [ ]:
from datetime import datetime
import time
import sys
import os
import json

import cmlapi

TARGET_RUNTIME = "ares-ecs-docker-repo.cloudera-field.org/cloudera/cdsw/ml-runtime-jupyterlab-python3.11-standard:2024.02.1-b4"
TARGET_PROJECT_ID = os.getenv("CDSW_PROJECT_ID")
MODEL_NAME = "sklearn_model"

# Set up client
workspace_domain = os.getenv("CDSW_DOMAIN")
cml_client = cmlapi.default_client(url=f"https://{workspace_domain}")

# Retrieve model id by model name
search_filter = {"model_name": MODEL_NAME}
response_dict = cml_client.list_registered_models(search_filter=json.dumps(search_filter)).to_dict()
model_id = response_dict["models"][0]["model_id"]
print(f"Model ID: {model_id}")

# Retrieve version id for deployment
registered_model = cml_client.get_registered_model(model_id, sort="-version_number")
model_version_id = registered_model.model_versions[0].model_version_id
print(f"Version ID: {model_version_id}")

Model ID: 60kx-qct5-oafw-zztv
Version ID: d0tw-sa2f-y0n6-lipz


In [44]:
# 1. Create Cloudera Machine Learning Model
CreateModelRequest = {
    "project_id": os.getenv("CDSW_PROJECT_ID"), 
    "name" : MODEL_NAME,
    "description": f"Production model deployment for model name: {MODEL_NAME}",
    "disable_authentication": True, # not recommended for production
    "registered_model_id": model_id
}

model_api_response = cml_client.create_model(CreateModelRequest, os.getenv("CDSW_PROJECT_ID"))

In [45]:
model_api_response

{'accelerator_label_id': '0',
 'access_key': 'm2j50mmrxoc33scw203lb5iag6mln65v',
 'auth_enabled': False,
 'created_at': datetime.datetime(2025, 2, 18, 16, 37, 8, 599233, tzinfo=tzutc()),
 'creator': {'email': 'mengelhardt@olympus.cloudera.com',
             'name': 'mengelhardt',
             'username': 'mengelhardt'},
 'crn': 'crn:cdp:ml:us-west-1:1f1c06d7-d1dd-4f61-9e01-0ac75c20312b:workspace:8bc314b2-d22f-4393-a660-434a27a3942e/bcc92b43-9545-4b64-8823-a3f00d26e1cf',
 'default_replication_policy': {'num_replicas': '1', 'type': 'fixed'},
 'default_resources': {'cpu_millicores': '1000',
                       'memory_mb': '2048',
                       'nvidia_gpus': '0'},
 'deletion_status': '',
 'description': 'Production model deployment for model name: sklearn_model',
 'id': 'bcc92b43-9545-4b64-8823-a3f00d26e1cf',
 'name': 'sklearn_model',
 'project': {'default_project_engine': '',
             'name': '',
             'public_identifier': '',
             'slug': ''},
 'registere

In [46]:
# 2. Create Cloudera Machine Learning Model Build
CreateModelBuildRequest = {
    "registered_model_version_id": model_version_id,
    "comment": "Invoking model build for model ",
    "runtime_identifier": TARGET_RUNTIME,
    "model_id": model_api_response.id
}

model_build_api_response = cml_client.create_model_build(CreateModelBuildRequest, TARGET_PROJECT_ID, model_api_response.id)

In [47]:
model_build_api_response

{'built_at': datetime.datetime(1, 1, 1, 0, 0, tzinfo=tzutc()),
 'comment': 'Invoking model build for model ',
 'created_at': datetime.datetime(2025, 2, 18, 16, 37, 8, 868910, tzinfo=tzutc()),
 'creator': {'email': 'mengelhardt@olympus.cloudera.com',
             'name': 'mengelhardt',
             'username': 'mengelhardt'},
 'crn': 'crn:cdp:ml:us-west-1:1f1c06d7-d1dd-4f61-9e01-0ac75c20312b:workspace:8bc314b2-d22f-4393-a660-434a27a3942e/cd14c841-edff-41b7-9b65-4e79b0a4c113',
 'deletion_status': '',
 'engine_image': '',
 'file_path': 'predict.py',
 'function_name': 'predict_with_metrics',
 'id': 'cd14c841-edff-41b7-9b65-4e79b0a4c113',
 'kernel': 'Python 3.11',
 'model_id': 'bcc92b43-9545-4b64-8823-a3f00d26e1cf',
 'registered_model_version_id': 'd0tw-sa2f-y0n6-lipz',
 'runtime_addon_identifiers': ['hadoop-cli-7.1.9.0-387'],
 'runtime_identifier': 'ares-ecs-docker-repo.cloudera-field.org/cloudera/cdsw/ml-runtime-jupyterlab-python3.11-standard:2024.02.1-b4',
 'status': 'pending',
 'updated

In [48]:
start_time = datetime.now()
print(start_time.strftime("%H:%M:%S"))

# keep track of the build process
while model_build_api_response.status not in ["built", "build failed"]:
    print("Waiting for model to build...")
    time.sleep(10)
    model_build_api_response = cml_client.get_model_build(
        TARGET_PROJECT_ID,
        model_api_response.id,
        model_build_api_response.id
    )
    if model_build_api_response.status == "build failed" :
        print("Model build failed, see UI for more information")
        sys.exit(1)

build_time = datetime.now()   
print(f"Time required for building model (sec): {(build_time - start_time).seconds}.")
print("Model build finished successfully.")

11:37:10
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Waiting for model to build...
Time required for building model (sec): 230.
Model build finished successfully.


In [ ]:
# 3. Create Cloudera Machine Learning Model Deployment
CreateModelDeploymentRequest = {
    "project_id": TARGET_PROJECT_ID,
    "model_id": model_api_response.id,
    "model_build_id": model_build_api_response.id,
    "cpu": 1,
    "memory": 2
}

model_deployment_api_response = cml_client.create_model_deployment(
    CreateModelDeploymentRequest,
    TARGET_PROJECT_ID,
    model_api_response.id,
    model_build_api_response.id
)

In [50]:
model_deployment_api_response

{'build_id': 'cd14c841-edff-41b7-9b65-4e79b0a4c113',
 'cpu': 1.0,
 'created_at': datetime.datetime(2025, 2, 18, 16, 41, 0, 382073, tzinfo=tzutc()),
 'crn': 'crn:cdp:ml:us-west-1:1f1c06d7-d1dd-4f61-9e01-0ac75c20312b:workspace:8bc314b2-d22f-4393-a660-434a27a3942e/9577c1cf-b037-4905-b9ca-00c67d16f576',
 'deployer': {'email': 'mengelhardt@olympus.cloudera.com',
              'name': 'mengelhardt',
              'username': 'mengelhardt'},
 'environment': '{}',
 'id': '9577c1cf-b037-4905-b9ca-00c67d16f576',
 'memory': 2.0,
 'model_id': 'bcc92b43-9545-4b64-8823-a3f00d26e1cf',
 'nvidia_gpu': 0,
 'project_id': 'x2ni-xczp-e1t4-5rqa',
 'replicas': 1,
 'status': 'deploying',
 'stopped_at': datetime.datetime(1, 1, 1, 0, 0, tzinfo=tzutc()),
 'updated_at': datetime.datetime(2025, 2, 18, 16, 41, 1, 986793, tzinfo=tzutc())}

In [ ]:
start_time = datetime.now()
print(start_time.strftime("%H:%M:%S"))

while model_deployment_api_response.status not in ["stopped", "failed", "deployed"]:
    print("Waiting for model to deploy...")
    time.sleep(10)
    model_deployment_api_response = cml_client.get_model_deployment(
        TARGET_PROJECT_ID,
        model_api_response.id,
        model_build_api_response.id,
        model_deployment_api_response.id
)

if model_deployment_api_response.status != "deployed":
    print("Model deployment failed, see UI for more information.")
    sys.exit(1)

if model_deployment_api_response.status == "deployed" :
    print(f"Time required for deploying model (sec): {(datetime.now() - start_time).seconds}.")
    print("Model deployed successfully!")

11:41:02
Waiting for model to deploy...
Waiting for model to deploy...
Waiting for model to deploy...
Waiting for model to deploy...
Waiting for model to deploy...
Time required for deploying model (sec): 50.
Model deployed successfully!
